In [1]:
storage_account_name = "<Storage account name>"
storage_account_key = "<Storage account key>"
container = "<Container name>"

In [2]:
dbutils.fs.mount(
 source = "wasbs://{0}@{1}.blob.core.windows.net".format(container, storage_account_name),
 mount_point = "/mnt/data",
 extra_configs = {"fs.azure.account.key.{0}.blob.core.windows.net".format(storage_account_name): storage_account_key}
)

--------------------------------------------------------------------------- 
 ExecutionError Traceback (most recent call last)
 <command-3600133196443986> in <module> () 
 2 source = "wasbs://{0}@{1}.blob.core.windows.net" . format ( container , storage_account_name ) , 
 3 mount_point = "/mnt/data" , 
 ----> 4 extra_configs = { "fs.azure.account.key.{0}.blob.core.windows.net" . format ( storage_account_name ) : storage_account_key } 
 5 )

 /local_disk0/tmp/1561932344064-0/dbutils.py in f_with_exception_handling (*args, **kwargs) 
 302 exc . __context__ = None 
 303 exc . __cause__ = None 
 --> 304 raise exc 
 305 return f_with_exception_handling 
 306 

 ExecutionError : An error occurred while calling o233.mount.
: java.rmi.RemoteException: java.lang.IllegalArgumentException: requirement failed: Directory already mounted: /mnt/data; nested exception is: 
	java.lang.IllegalArgumentException: requirement failed: Directory already mounted: /mnt/data
	at com.databricks.backend.daemon.data.client.DbfsClient.send0(DbfsClient.scala:100)
	at com.databricks.backend.daemon.data.client.DbfsClient.sendIdempotent(DbfsClient.scala:54)
	at com.databricks.backend.daemon.dbutils.DBUtilsCore.mount(DBUtilsCore.scala:454)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:380)
	at py4j.Gateway.invoke(Gateway.java:295)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:251)
	at java.lang.Thread.run(Thread.java:748)
Caused by: java.lang.IllegalArgumentException: requirement failed: Directory already mounted: /mnt/data
	at scala.Predef$.require(Predef.scala:281)
	at com.databricks.backend.daemon.data.server.DefaultMetadataManager.$anonfun$insertMount$1(MetadataManager.scala:160)
	at com.databricks.backend.daemon.data.server.DefaultMetadataManager.withRetries(MetadataManager.scala:206)
	at com.databricks.backend.daemon.data.server.DefaultMetadataManager.insertMount(MetadataManager.scala:154)
	at com.databricks.backend.daemon.data.server.handler.MountHandler.receive(MountHandler.scala:79)
	at com.databricks.backend.daemon.data.server.session.SessionContext.$anonfun$queryHandlers$1(SessionContext.scala:103)
	at com.databricks.backend.daemon.data.server.session.SessionContext.$anonfun$queryHandlers$1$adapted(SessionContext.scala:102)
	at scala.collection.immutable.List.foreach(List.scala:392)
	at com.databricks.backend.daemon.data.server.session.SessionContext.queryHandlers(SessionContext.scala:102)
	at com.databricks.backend.daemon.data.server.DbfsServerBackend$$anonfun$receive$3.applyOrElse(DbfsServerBackend.scala:298)
	at com.databricks.backend.daemon.data.server.DbfsServerBackend$$anonfun$receive$3.applyOrElse(DbfsServerBackend.scala:276)
	at com.databricks.rpc.ServerBackend.$anonfun$internalReceive$2(ServerBackend.scala:44)
	at com.databricks.rpc.ServerBackend$$anonfun$commonReceive$1.applyOrElse(ServerBackend.scala:61)
	at com.databricks.rpc.ServerBackend$$anonfun$commonReceive$1.applyOrElse(ServerBackend.scala:61)
	at com.databricks.rpc.ServerBackend.$anonfun$internalReceive$1(ServerBackend.scala:40)
	at com.databricks.logging.UsageLogging.$anonfun$recordOperation$2(UsageLogging.scala:369)
	at com.databricks.logging.UsageLogging.$anonfun$withAttributionContext$1(UsageLogging.scala:238)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:62)
	at com.databricks.logging.UsageLogging.withAttributionContext(UsageLogging.scala:233)
	at com.databricks.logging.UsageLogging.withAttributionContext$(UsageLogging.scala:230)
	at com.databricks.rpc.Ser

In [3]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [4]:
data = spark.read \
  .option("header", "true") \
  .option("inferSchema", "true") \
  .option("delimiter", ",") \
  .csv("/mnt/data/heart.csv".format(container, storage_account_name))

data.show()

+---+---+---+--------+----+---+-------+-------+-----+-------+-----+---+----+------+
age|sex| cp|trestbps|chol|fbs|restecg|thalach|exang|oldpeak|slope| ca|thal|target|
+---+---+---+--------+----+---+-------+-------+-----+-------+-----+---+----+------+
 63| 1| 3| 145| 233| 1| 0| 150| 0| 2.3| 0| 0| 1| 1|
 37| 1| 2| 130| 250| 0| 1| 187| 0| 3.5| 0| 0| 2| 1|
 41| 0| 1| 130| 204| 0| 0| 172| 0| 1.4| 2| 0| 2| 1|
 56| 1| 1| 120| 236| 0| 1| 178| 0| 0.8| 2| 0| 2| 1|
 57| 0| 0| 120| 354| 0| 1| 163| 1| 0.6| 2| 0| 2| 1|
 57| 1| 0| 140| 192| 0| 1| 148| 0| 0.4| 1| 0| 1| 1|
 56| 0| 1| 140| 294| 0| 0| 153| 0| 1.3| 1| 0| 2| 1|
 44| 1| 1| 120| 263| 0| 1| 173| 0| 0.0| 2| 0| 3| 1|
 52| 1| 2| 172| 199| 1| 1| 162| 0| 0.5| 2| 0| 3| 1|
 57| 1| 2| 150| 168| 0| 1| 174| 0| 1.6| 2| 0| 2| 1|
 54| 1| 0| 140| 239| 0| 1| 160| 0| 1.2| 2| 0| 2| 1|
 48| 0| 2| 130| 275| 0| 1| 139| 0| 0.2| 2| 0| 2| 1|
 49| 1| 1| 130| 266| 0| 1| 171| 0| 0.6| 2| 0| 2| 1|
 64| 1| 3| 110| 211| 0| 0| 144| 1| 1.8| 1| 0| 2| 1|
 58| 0| 3| 150| 283| 1| 0| 162| 0| 1.0| 2| 0| 2| 1|
 50| 0| 2| 120| 219| 0| 1| 158| 0| 1.6| 1| 0| 2| 1|
 58| 0| 2| 120| 340| 0| 1| 172| 0| 0.0| 2| 0| 2| 1|
 66| 0| 3| 150| 226| 0| 1| 114| 0| 2.6| 0| 0| 2| 1|
 43| 1| 0| 150| 247| 0| 1| 171| 0| 1.5| 2| 0| 2| 1|
 69| 0| 3| 140| 239| 0| 1| 151| 0| 1.8| 2| 2| 2| 1|
+---+---+---+--------+----+---+-------+-------+-----+-------+-----+---+----+------+
only showing top 20 rows

In [5]:
feature_inputs = ["age", "sex", "cp", "trestbps", "chol", "fbs", "restecg", "thalach", "exang", "oldpeak", "slope", "ca", "thal"]

vectors = VectorAssembler(inputCols=feature_inputs, outputCol="features")

In [6]:
vector_data = vectors.transform(data)

features = vector_data.select(["features", "target"])

In [7]:
features = features.withColumnRenamed("target", "label")

In [8]:
features = features.withColumn("label", features["label"].cast("double"))

In [9]:
(training_data, test_data) = features.randomSplit([0.7, 0.3])

In [10]:
lr = LogisticRegression(labelCol="label", featuresCol="features")

In [11]:
model = lr.fit(training_data)

In [12]:
model_summary = model.summary

print("AUC:", model_summary.areaUnderROC)

AUC: 0.9239766081871345

In [13]:
evaluator = BinaryClassificationEvaluator()

In [14]:
predictions = model.transform(test_data)

In [15]:
evaluator.evaluate(predictions)

Out[ 76 ]: 0.9215686274509803

In [16]:
display(model, training_data, "ROC")

False Positive Rate,True Positive Rate,Threshold
0.0,0.0,0.9964419552239666
0.0,0.01818181818181818,0.9964419552239666
0.0,0.03636363636363636,0.9961240845650033
0.0,0.05454545454545454,0.9957672945091514
0.0,0.07272727272727272,0.9954209825330019
0.0,0.09090909090909091,0.9943105379067675
0.0,0.10909090909090909,0.9927084164294001
0.0,0.12727272727272726,0.9907005934637279
0.0,0.14545454545454545,0.9896445959960192
0.0,0.16363636363636364,0.982317581925775
